In [1]:
import os
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
# from tensorflow.keras.layers.experimental.preprocessing import *
from tensorflow.keras.optimizers import SGD, Adam
from keras.losses import BinaryCrossentropy
from keras.regularizers import l2

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
DATA_DIR = 'data'
HDF5_FILE = 'data_tf.hdf5'

In [4]:
BATCH_SIZE = 64
IMG_SHAPE = (96, 96, 4)

In [5]:
data_file = os.path.join(DATA_DIR, HDF5_FILE)
ds_x = tfio.IODataset.from_hdf5(data_file, '/x_train')
ds_y = tfio.IODataset.from_hdf5(data_file, '/y_train')

In [6]:
# convert to float and scale to [0, 1)
ds_x = ds_x.map(lambda x: tf.image.convert_image_dtype(x, tf.float32))
# ds_x = ds_x.map(lambda x: tf.image.crop_to_bounding_box(x, 8, 8, 80, 80))
# ds_x = ds_images.map(lambda x: tf.image.per_image_standardization(x))
ds_xc = ds_x.map(lambda x: tf.add(x, tf.random.normal(tf.shape(x), 0, 0.2)))

In [7]:
ds_xy = tf.data.Dataset.zip((ds_x, ds_y))
ds_xy = ds_xy.shuffle(BATCH_SIZE*2)

ds_xcy = tf.data.Dataset.zip((ds_xc, ds_y))
ds_xcy = ds_xcy.shuffle(BATCH_SIZE*2)

In [8]:
# ds_len = ds_xy.reduce(0, lambda x, _: x + 1).numpy()
ds_len = 220025
train_len = int(0.9 * ds_len)

ds_train = ds_xy.take(train_len)
ds_trainc = ds_xcy.take(train_len)
ds_test = ds_xy.skip(train_len)

ds_len, train_len

(220025, 198022)

In [9]:
from keras.utils import control_flow_util
from keras.layers.preprocessing.image_preprocessing import transform, get_rotation_matrix
import numpy as np

H_AXIS = -3
W_AXIS = -2

class RandomRot90(tf.keras.layers.RandomRotation):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def call(self, inputs, training=True):
        if training is None:
            training = backend.learning_phase()

        inputs = tf.convert_to_tensor(inputs)
        original_shape = inputs.shape
        unbatched = inputs.shape.rank == 3
        # The transform op only accepts rank 4 inputs, so if we have an unbatched
        # image, we need to temporarily expand dims to a batch.
        if unbatched:
            inputs = tf.expand_dims(inputs, 0)

        def random_rotated_inputs():
            """Rotated inputs with random ops."""
            inputs_shape = tf.shape(inputs)
            batch_size = inputs_shape[0]
            img_hd = tf.cast(inputs_shape[H_AXIS], tf.float32)
            img_wd = tf.cast(inputs_shape[W_AXIS], tf.float32)
            #min_angle = self.lower * 2. * np.pi
            #max_angle = self.upper * 2. * np.pi
            angles = self._rng.uniform(
                shape=[batch_size], minval=self.lower, maxval=self.upper)            
            angles = tf.math.floordiv(angles, 0.25)
            angles = tf.multiply(angles, 0.5 * np.pi)
            return transform(
                inputs,
                get_rotation_matrix(angles, img_hd, img_wd),
                fill_mode=self.fill_mode,
                fill_value=self.fill_value,
                interpolation=self.interpolation)

        output = control_flow_util.smart_cond(training, random_rotated_inputs,
                                              lambda: inputs)
        if unbatched:
              output = tf.squeeze(output, 0)
        output.set_shape(original_shape)
        return output

In [10]:
data_aug = Sequential([
    RandomFlip("horizontal_and_vertical"),
    RandomRot90((0., 1.), fill_mode='constant')
    # RandomZoom(0.2, 0.2)
])

In [11]:
model = Sequential()

model.add(data_aug)

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4))) #, input_shape=(X_train.shape[1:])))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D (pool_size=(2, 2)))
model.add(Dropout(0.15))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D (pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# model.summary()

In [12]:
ds_trainc = ds_trainc.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-3),
              metrics=['accuracy'])

history = model.fit(ds_trainc,
                    epochs=8,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/8
3095/3095 - 158s - loss: 0.4846 - accuracy: 0.8165 - val_loss: 0.7085 - val_accuracy: 0.7270
Epoch 2/8
3095/3095 - 154s - loss: 0.3512 - accuracy: 0.8673 - val_loss: 0.6130 - val_accuracy: 0.7875
Epoch 3/8
3095/3095 - 154s - loss: 0.3077 - accuracy: 0.8898 - val_loss: 0.8992 - val_accuracy: 0.7449
Epoch 4/8
3095/3095 - 154s - loss: 0.2922 - accuracy: 0.8976 - val_loss: 0.9055 - val_accuracy: 0.7564
Epoch 5/8
3095/3095 - 154s - loss: 0.2856 - accuracy: 0.9020 - val_loss: 1.1763 - val_accuracy: 0.7523
Epoch 6/8
3095/3095 - 154s - loss: 0.2803 - accuracy: 0.9039 - val_loss: 0.6371 - val_accuracy: 0.7912
Epoch 7/8
3095/3095 - 154s - loss: 0.2762 - accuracy: 0.9062 - val_loss: 0.7589 - val_accuracy: 0.7964
Epoch 8/8
3095/3095 - 154s - loss: 0.2733 - accuracy: 0.9077 - val_loss: 0.7881 - val_accuracy: 0.7845


In [13]:
ds_train = ds_train.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

history = model.fit(ds_train,
                    epochs=8,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/8
3095/3095 - 154s - loss: 0.2257 - accuracy: 0.9300 - val_loss: 0.3018 - val_accuracy: 0.9188
Epoch 2/8
3095/3095 - 153s - loss: 0.2168 - accuracy: 0.9337 - val_loss: 1.0304 - val_accuracy: 0.7823
Epoch 3/8
3095/3095 - 153s - loss: 0.2136 - accuracy: 0.9350 - val_loss: 0.4646 - val_accuracy: 0.8511
Epoch 4/8
3095/3095 - 154s - loss: 0.2106 - accuracy: 0.9364 - val_loss: 0.3104 - val_accuracy: 0.8852
Epoch 5/8
3095/3095 - 153s - loss: 0.2091 - accuracy: 0.9372 - val_loss: 0.6924 - val_accuracy: 0.8023
Epoch 6/8
3095/3095 - 153s - loss: 0.2074 - accuracy: 0.9382 - val_loss: 0.2589 - val_accuracy: 0.9146
Epoch 7/8
3095/3095 - 153s - loss: 0.2067 - accuracy: 0.9387 - val_loss: 0.3367 - val_accuracy: 0.8952
Epoch 8/8
3095/3095 - 153s - loss: 0.2044 - accuracy: 0.9395 - val_loss: 0.2735 - val_accuracy: 0.9129


In [14]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(128).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(128).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=2e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=8,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/8
1548/1548 - 154s - loss: 0.1673 - accuracy: 0.9534 - val_loss: 0.2605 - val_accuracy: 0.9227
Epoch 2/8
1548/1548 - 151s - loss: 0.1575 - accuracy: 0.9562 - val_loss: 0.1511 - val_accuracy: 0.9596
Epoch 3/8
1548/1548 - 151s - loss: 0.1526 - accuracy: 0.9575 - val_loss: 0.1727 - val_accuracy: 0.9496
Epoch 4/8
1548/1548 - 151s - loss: 0.1497 - accuracy: 0.9580 - val_loss: 0.2010 - val_accuracy: 0.9399
Epoch 5/8
1548/1548 - 151s - loss: 0.1454 - accuracy: 0.9590 - val_loss: 0.1976 - val_accuracy: 0.9416
Epoch 6/8
1548/1548 - 151s - loss: 0.1432 - accuracy: 0.9597 - val_loss: 0.1997 - val_accuracy: 0.9382
Epoch 7/8
1548/1548 - 151s - loss: 0.1414 - accuracy: 0.9601 - val_loss: 0.1379 - val_accuracy: 0.9611
Epoch 8/8
1548/1548 - 151s - loss: 0.1396 - accuracy: 0.9602 - val_loss: 0.1755 - val_accuracy: 0.9442


In [15]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(256).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(256).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=8,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/8
774/774 - 156s - loss: 0.1292 - accuracy: 0.9640 - val_loss: 0.1238 - val_accuracy: 0.9670
Epoch 2/8
774/774 - 151s - loss: 0.1285 - accuracy: 0.9644 - val_loss: 0.1224 - val_accuracy: 0.9672
Epoch 3/8
774/774 - 151s - loss: 0.1269 - accuracy: 0.9647 - val_loss: 0.1213 - val_accuracy: 0.9677
Epoch 4/8
774/774 - 152s - loss: 0.1270 - accuracy: 0.9646 - val_loss: 0.1206 - val_accuracy: 0.9677
Epoch 5/8
774/774 - 152s - loss: 0.1256 - accuracy: 0.9651 - val_loss: 0.1202 - val_accuracy: 0.9680
Epoch 6/8
774/774 - 152s - loss: 0.1246 - accuracy: 0.9654 - val_loss: 0.1230 - val_accuracy: 0.9663
Epoch 7/8
774/774 - 151s - loss: 0.1241 - accuracy: 0.9650 - val_loss: 0.1236 - val_accuracy: 0.9666
Epoch 8/8
774/774 - 152s - loss: 0.1240 - accuracy: 0.9657 - val_loss: 0.1172 - val_accuracy: 0.9686


In [16]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(64).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(64).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=5,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/5
3095/3095 - 154s - loss: 0.1393 - accuracy: 0.9596 - val_loss: 0.1546 - val_accuracy: 0.9511
Epoch 2/5
3095/3095 - 153s - loss: 0.1371 - accuracy: 0.9601 - val_loss: 0.1290 - val_accuracy: 0.9630
Epoch 3/5
3095/3095 - 153s - loss: 0.1365 - accuracy: 0.9600 - val_loss: 0.1507 - val_accuracy: 0.9550
Epoch 4/5
3095/3095 - 153s - loss: 0.1356 - accuracy: 0.9602 - val_loss: 0.1244 - val_accuracy: 0.9645
Epoch 5/5
3095/3095 - 153s - loss: 0.1341 - accuracy: 0.9608 - val_loss: 0.1295 - val_accuracy: 0.9624


In [17]:
history = model.fit(ds_trainc,
                    epochs=2,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/2
3095/3095 - 153s - loss: 0.2232 - accuracy: 0.9214 - val_loss: 2.1564 - val_accuracy: 0.7285
Epoch 2/2
3095/3095 - 153s - loss: 0.2088 - accuracy: 0.9283 - val_loss: 1.8633 - val_accuracy: 0.7327


In [18]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(256).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(256).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=5,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/5
774/774 - 152s - loss: 0.1356 - accuracy: 0.9595 - val_loss: 0.1260 - val_accuracy: 0.9640
Epoch 2/5
774/774 - 151s - loss: 0.1260 - accuracy: 0.9637 - val_loss: 0.1198 - val_accuracy: 0.9666
Epoch 3/5
774/774 - 152s - loss: 0.1231 - accuracy: 0.9650 - val_loss: 0.1177 - val_accuracy: 0.9667
Epoch 4/5
774/774 - 152s - loss: 0.1203 - accuracy: 0.9659 - val_loss: 0.1153 - val_accuracy: 0.9677
Epoch 5/5
774/774 - 152s - loss: 0.1190 - accuracy: 0.9663 - val_loss: 0.1176 - val_accuracy: 0.9677


In [19]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(512).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(512).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=2,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/2
387/387 - 162s - loss: 0.1260 - accuracy: 0.9638 - val_loss: 0.1485 - val_accuracy: 0.9561
Epoch 2/2
387/387 - 153s - loss: 0.1244 - accuracy: 0.9640 - val_loss: 0.1689 - val_accuracy: 0.9481


In [20]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=5,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/5
387/387 - 154s - loss: 0.1161 - accuracy: 0.9672 - val_loss: 0.1166 - val_accuracy: 0.9681
Epoch 2/5
387/387 - 153s - loss: 0.1135 - accuracy: 0.9684 - val_loss: 0.1114 - val_accuracy: 0.9690
Epoch 3/5
387/387 - 153s - loss: 0.1128 - accuracy: 0.9683 - val_loss: 0.1116 - val_accuracy: 0.9694
Epoch 4/5
387/387 - 153s - loss: 0.1119 - accuracy: 0.9683 - val_loss: 0.1111 - val_accuracy: 0.9692
Epoch 5/5
387/387 - 153s - loss: 0.1110 - accuracy: 0.9688 - val_loss: 0.1116 - val_accuracy: 0.9694


In [21]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_trainc,
                    epochs=2,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/2
3095/3095 - 153s - loss: 0.2198 - accuracy: 0.9227 - val_loss: 2.9547 - val_accuracy: 0.7058
Epoch 2/2
3095/3095 - 152s - loss: 0.2033 - accuracy: 0.9292 - val_loss: 2.2697 - val_accuracy: 0.7373


In [22]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=5,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/5
387/387 - 154s - loss: 0.1372 - accuracy: 0.9580 - val_loss: 0.1240 - val_accuracy: 0.9651
Epoch 2/5
387/387 - 153s - loss: 0.1242 - accuracy: 0.9635 - val_loss: 0.1197 - val_accuracy: 0.9661
Epoch 3/5
387/387 - 153s - loss: 0.1215 - accuracy: 0.9645 - val_loss: 0.1174 - val_accuracy: 0.9666
Epoch 4/5
387/387 - 153s - loss: 0.1186 - accuracy: 0.9660 - val_loss: 0.1159 - val_accuracy: 0.9675
Epoch 5/5
387/387 - 153s - loss: 0.1172 - accuracy: 0.9662 - val_loss: 0.1138 - val_accuracy: 0.9680


In [23]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=2,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/2
387/387 - 154s - loss: 0.1156 - accuracy: 0.9671 - val_loss: 0.1202 - val_accuracy: 0.9658
Epoch 2/2
387/387 - 153s - loss: 0.1129 - accuracy: 0.9682 - val_loss: 0.1106 - val_accuracy: 0.9694


In [24]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=5,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/5
387/387 - 153s - loss: 0.1105 - accuracy: 0.9687 - val_loss: 0.1089 - val_accuracy: 0.9696
Epoch 2/5
387/387 - 153s - loss: 0.1102 - accuracy: 0.9687 - val_loss: 0.1089 - val_accuracy: 0.9696
Epoch 3/5
387/387 - 153s - loss: 0.1099 - accuracy: 0.9693 - val_loss: 0.1083 - val_accuracy: 0.9703
Epoch 4/5
387/387 - 153s - loss: 0.1100 - accuracy: 0.9691 - val_loss: 0.1082 - val_accuracy: 0.9700
Epoch 5/5
387/387 - 153s - loss: 0.1091 - accuracy: 0.9694 - val_loss: 0.1082 - val_accuracy: 0.9696


In [ ]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=SGD(learning_rate=1e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=5,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/5
387/387 - 154s - loss: 0.1096 - accuracy: 0.9691 - val_loss: 0.1089 - val_accuracy: 0.9697
Epoch 2/5
387/387 - 153s - loss: 0.1092 - accuracy: 0.9691 - val_loss: 0.1090 - val_accuracy: 0.9696
Epoch 3/5
